## Installing dependencies

In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib
!pip install torchdiffeq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.8

**Download the dataset**

In [ ]:
# !rm -r Pems_Dataset/
!git clone https://github.com/ebagirma/Pems_Dataset.git
%cd Pems_Dataset
!ls

Cloning into 'Pems_Dataset'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 44 (delta 9), reused 41 (delta 6), pack-reused 0
Unpacking objects: 100% (44/44), 31.72 MiB | 8.60 MiB/s, done.
/content/Pems_Dataset
models	PEMS04	pems04_dtw_distance.npy  pems04_spatial_distance.npy  README.md


In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from Pems_Dataset.models.model import ODEGCN
# from Pems_Dataset.models.Update import LocalUpdate
from torch.optim.lr_scheduler import StepLR, OneCycleLR

import copy
import argparse
import os
import csv
import numpy as np
from fastdtw import fastdtw
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch


from torch.utils.data import DataLoader, random_split


import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 1.13.1+cu116 and Flower 1.3.0


# Load arguments

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument('--remote', action='store_true', help='the code run on a server')
parser.add_argument('--num-gpu', type=int, default=0, help='the number of the gpu to use')
parser.add_argument('--epochs', type=int, default=1, help='train epochs')
parser.add_argument('--batch-size', type=int, default=16, help='batch size')
parser.add_argument('--batch', type=int, default=16, help='batch size')


parser.add_argument('--frac', type=float, default=0.01, help="the fraction of clients: C")
parser.add_argument('--num_users', type=int, default=100)


parser.add_argument('--filename', type=str, default='pems04')
parser.add_argument('--train-ratio', type=float, default=0.6, help='the ratio of training dataset')
parser.add_argument('--valid-ratio', type=float, default=0.2, help='the ratio of validating dataset')
parser.add_argument('--his-length', type=int, default=12, help='the length of history time series of input')
parser.add_argument('--pred-length', type=int, default=12, help='the length of target time series for prediction')

parser.add_argument('--sigma1', type=float, default=0.1, help='sigma for the semantic matrix')
parser.add_argument('--sigma2', type=float, default=10, help='sigma for the spatial matrix')
parser.add_argument('--thres1', type=float, default=0.6, help='the threshold for the semantic matrix')
parser.add_argument('--thres2', type=float, default=0.5, help='the threshold for the spatial matrix')
parser.add_argument('--lr', type=float, default=2e-3, help='learning rate')

parser.add_argument('--log', action='store_true', help='if write log to files')
args, unkown = parser.parse_known_args()

In [ ]:
files = {
    'pems03': ['PEMS03/pems03.npz', 'PEMS03/distance.csv'],
    'pems04': ['PEMS04/PEMS04.npz', 'PEMS04/distance.csv'],
    'pems07': ['PEMS07/pems07.npz', 'PEMS07/distance.csv'],
    'pems08': ['PEMS08/pems08.npz', 'PEMS08/distance.csv'],
    'pemsbay': ['PEMSBAY/pems_bay.npz', 'PEMSBAY/distance.csv'],
    'pemsD7M': ['PeMSD7M/PeMSD7M.npz', 'PeMSD7M/distance.csv'],
    'pemsD7L': ['PeMSD7L/PeMSD7L.npz', 'PeMSD7L/distance.csv']
}

In [ ]:
%cd /content/
def read_data(args):
    """read data, generate spatial adjacency matrix and semantic adjacency matrix by dtw
    Args:
        sigma1: float, default=0.1, sigma for the semantic matrix
        sigma2: float, default=10, sigma for the spatial matrix
        thres1: float, default=0.6, the threshold for the semantic matrix
        thres2: float, default=0.5, the threshold for the spatial matrix
    Returns:
        data: tensor, T * N * 1
        dtw_matrix: array, semantic adjacency matrix
        sp_matrix: array, spatial adjacency matrix
    """
    filename = args.filename
    file = files[filename]
    filepath = "./Pems_Dataset/"
    if args.remote:
        filepath = './Pems_Dataset/'
    data = np.load(filepath + file[0])['data']
    # PEMS04 == shape: (16992, 307, 3)    feature: flow,occupy,speed
    # PEMSD7M == shape: (12672, 228, 1)
    # PEMSD7L == shape: (12672, 1026, 1)
    num_node = data.shape[1]
    mean_value = np.mean(data, axis=(0, 1)).reshape(1, 1, -1)
    std_value = np.std(data, axis=(0, 1)).reshape(1, 1, -1)
    data = (data - mean_value) / std_value
    mean_value = mean_value.reshape(-1)[0]
    std_value = std_value.reshape(-1)[0]

    if not os.path.exists(f'Pems_Dataset/{filename}_dtw_distance.npy'):
        data_mean = np.mean([data[:, :, 0][24*12*i: 24*12*(i+1)] for i in range(data.shape[0]//(24*12))], axis=0)
        data_mean = data_mean.squeeze().T 
        dtw_distance = np.zeros((num_node, num_node))
        for i in tqdm(range(num_node)):
            for j in range(i, num_node):
                dtw_distance[i][j] = fastdtw(data_mean[i], data_mean[j], radius=6)[0]
        for i in range(num_node):
            for j in range(i):
                dtw_distance[i][j] = dtw_distance[j][i]
        np.save(f'data/{filename}_dtw_distance.npy', dtw_distance)

    dist_matrix = np.load(f'Pems_Dataset/{filename}_dtw_distance.npy')

    mean = np.mean(dist_matrix)
    std = np.std(dist_matrix)
    dist_matrix = (dist_matrix - mean) / std
    sigma = args.sigma1
    dist_matrix = np.exp(-dist_matrix ** 2 / sigma ** 2)
    dtw_matrix = np.zeros_like(dist_matrix)
    dtw_matrix[dist_matrix > args.thres1] = 1

    # use continuous spatial matrix
    if not os.path.exists(f'Pems_Dataset/{filename}_spatial_distance.npy'):
        with open(filepath + file[1], 'r') as fp:
            dist_matrix = np.zeros((num_node, num_node)) + np.float('inf')
            file = csv.reader(fp)
            for line in file:
                break
            for line in file:
                start = int(line[0])
                end = int(line[1])
                dist_matrix[start][end] = float(line[2])
                dist_matrix[end][start] = float(line[2])
            np.save(f'Pems_Dataset/{filename}_spatial_distance.npy', dist_matrix)


    dist_matrix = np.load(f'Pems_Dataset/{filename}_spatial_distance.npy')
    # normalization
    std = np.std(dist_matrix[dist_matrix != np.float('inf')])
    mean = np.mean(dist_matrix[dist_matrix != np.float('inf')])
    dist_matrix = (dist_matrix - mean) / std
    sigma = args.sigma2
    sp_matrix = np.exp(- dist_matrix**2 / sigma**2)
    sp_matrix[sp_matrix < args.thres2] = 0 


    print(f'average degree of spatial graph is {np.sum(sp_matrix > 0)/2/num_node}')
    print(f'average degree of semantic graph is {np.sum(dtw_matrix > 0)/2/num_node}')
    return torch.from_numpy(data.astype(np.float32)), mean_value, std_value, dtw_matrix, sp_matrix


def get_normalized_adj(A):
    """
    Returns a tensor, the degree normalized adjacency matrix.
    """
    alpha = 0.8
    D = np.array(np.sum(A, axis=1)).reshape((-1,))
    D[D <= 10e-5] = 10e-5    # Prevent infs
    diag = np.reciprocal(np.sqrt(D))
    A_wave = np.multiply(np.multiply(diag.reshape((-1, 1)), A),
                         diag.reshape((1, -1)))
    A_reg = alpha / 2 * (np.eye(A.shape[0]) + A_wave)
    return torch.from_numpy(A_reg.astype(np.float32))


class MyDataset(Dataset):
    def __init__(self, data, split_start, split_end, his_length, pred_length):
        split_start = int(split_start)
        split_end = int(split_end)
        self.data = data[split_start: split_end]
        self.his_length = his_length
        self.pred_length = pred_length
    
    def __getitem__(self, index):
        x = self.data[index: index + self.his_length].permute(1, 0, 2)
        y = self.data[index + self.his_length: index + self.his_length + self.pred_length][:, :, 0].permute(1, 0)
        return torch.Tensor(x), torch.Tensor(y)
    def __len__(self):
        return self.data.shape[0] - self.his_length - self.pred_length + 1


def generate_dataset(data, args):
    """
    Args:
        data: input dataset, shape like T * N
        batch_size: int 
        train_ratio: float, the ratio of the dataset for training
        his_length: the input length of time series for prediction
        pred_length: the target length of time series of prediction
    Returns:
        train_dataloader: torch tensor, shape like batch * N * his_length * features
        test_dataloader: torch tensor, shape like batch * N * pred_length * features
    """
    batch_size = args.batch_size
    train_ratio = args.train_ratio
    valid_ratio = args.valid_ratio
    his_length = args.his_length
    pred_length = args.pred_length
    train_dataset = MyDataset(data, 0, data.shape[0] * train_ratio, his_length, pred_length)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MyDataset(data, data.shape[0]*train_ratio, data.shape[0]*(train_ratio+valid_ratio), his_length, pred_length)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = MyDataset(data, data.shape[0]*(train_ratio+valid_ratio), data.shape[0], his_length, pred_length)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, valid_dataloader, test_dataloader


/content


# Load the dataset

In [ ]:
data, mean, std, dtw_matrix, sp_matrix = read_data(args)
train_dataset, valloaders, testloader = generate_dataset(data, args)

average degree of spatial graph is 1.1009771986970684
average degree of semantic graph is 6.267100977198697


<ipython-input-6-04a0e5f37a81>:69: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  std = np.std(dist_matrix[dist_matrix != np.float('inf')])
<ipython-input-6-04a0e5f37a81>:70: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mean = np.mean(dist_matrix[dist_matrix != np.float('inf')])


**Hyper-parameters**

In [ ]:
# Normalize Adjacency matrix
A_sp_wave = get_normalized_adj(sp_matrix).to(DEVICE)
A_se_wave = get_normalized_adj(dtw_matrix).to(DEVICE)

In [ ]:
import numpy as np

def divide_iid(dataset, num_users):
    """
    Sample I.I.D. client data from PEMS03 dataset
    :param dataset:
    :param num_users:
    :return: dict of data index 
    """
    
    # I.e, here the length of the training dataset is 10172 and
    # the number of user is 100, so the each dict user will have 10172/100 => 101
    # 101 will be a the length of the dataset, which is dived for the 100 local users. 
    
    num_items = int(len(dataset)/num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        dict_users[i] = set(np.random.choice(all_idxs, num_items, replace=False))
        all_idxs = list(set(all_idxs) - dict_users[i])
    return dict_users
    
dict_users = divide_iid(train_dataset, args.num_users)

In [ ]:
net_glob = ODEGCN(num_nodes=data.shape[1], 
                  num_features=data.shape[2], 
                  num_timesteps_input=args.his_length, 
                  num_timesteps_output=args.pred_length, 
                  A_sp_hat=A_sp_wave, 
                  A_se_hat=A_se_wave)

In [ ]:
lr = args.lr
optimizer = torch.optim.AdamW(net_glob.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

In [ ]:
# Set the model to train and send it to device.
net_glob = net_glob.to(DEVICE)
net_glob.train()


# copy weights
w_glob = net_glob.state_dict() #Get network parameters
 

In [ ]:
# training 
loss_train = []
acc_train = []

In [ ]:
import copy
import torch

def FedAvg(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg

In [ ]:
import torch
from torch import nn, autograd
from torch.utils.data import DataLoader, Dataset





class DatasetSplit(Dataset):
    """
    Splits the datasets by the idxs
    """
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        input, label = self.dataset[self.idxs[item]]
        return input, label


class LocalUpdate(object):
    """
    Model Aggregation (Federated Averaging)
    """
    def __init__(self, args, dataset=None, idxs=None):
        """
        Args:
            args: contains arguments passeds
            dataset: the training dataset 
            idxs: index of the dict users
        """
        self.args = args
        self.loss_func = nn.SmoothL1Loss()                         # It is less sensitive to outliers and prevents exploding gradients 
        self.selected_clients = []
        self.ldr_train = DataLoader(DatasetSplit(dataset, idxs), batch_size=self.args.batch, shuffle=True)
        print(type( self.ldr_train))
    def train(self, net):
        net.train()
        # train and update
        optimizer = torch.optim.SGD(net.parameters(), lr=self.args.lr, momentum=0.5)

        epoch_loss = []
        for iter in range(self.args.epochs):
            
            batch_loss = []
            for batch_idx, (inputs, labels) in enumerate(self.ldr_train):
                
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                
                net.zero_grad()                                   #  Sets the gradients of all its parameters to zero for the local paramaters to learn new values
                log_probs = net(inputs)
                
                # print(inputs.shape)
                # print(log_probs.shape)
                
                loss = self.loss_func(log_probs, labels)
                loss.backward()
                optimizer.step()
                
                
                ## -------------------------- Prints the steps in each epoch ---------------------------- #
                
                print('Update Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    iter, batch_idx * len(inputs), len(self.ldr_train.dataset),
                            100. * batch_idx / len(self.ldr_train), loss.item()))
                
                
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            
        return net.state_dict(), sum(epoch_loss) / len(epoch_loss)

In [ ]:

def mask_np(array, null_val):
    if np.isnan(null_val):
        return (~np.isnan(null_val)).astype('float32')
    else:
        return np.not_equal(array, null_val).astype('float32')


def masked_mape_np(y_true, y_pred, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        mask = mask_np(y_true, null_val)
        mask /= mask.mean()
        mape = np.abs((y_pred - y_true) / y_true)
        mape = np.nan_to_num(mask * mape)
        return np.mean(mape) * 100


def masked_rmse_np(y_true, y_pred, null_val=np.nan):
    mask = mask_np(y_true, null_val)
    mask /= mask.mean()
    mse = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(np.nan_to_num(mask * mse)))


def masked_mae_np(y_true, y_pred, null_val=np.nan):
    mask = mask_np(y_true, null_val)
    mask /= mask.mean()
    mae = np.abs(y_true - y_pred)
    return np.mean(np.nan_to_num(mask * mae))

In [ ]:
@torch.no_grad()
def eval(loader, model, std, mean, device):
    batch_rmse_loss = 0  
    batch_mae_loss = 0
    batch_mape_loss = 0
    for idx, (inputs, targets) in enumerate(tqdm(loader)):
        model.eval()

        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        
        out_unnorm = output.detach().cpu().numpy()*std + mean
        target_unnorm = targets.detach().cpu().numpy()*std + mean

        mae_loss = masked_mae_np(target_unnorm, out_unnorm, 0)
        rmse_loss = masked_rmse_np(target_unnorm, out_unnorm, 0)
        mape_loss = masked_mape_np(target_unnorm, out_unnorm, 0)
        batch_rmse_loss += rmse_loss
        batch_mae_loss += mae_loss
        batch_mape_loss += mape_loss

    return batch_rmse_loss / (idx + 1), batch_mae_loss / (idx + 1), batch_mape_loss / (idx + 1)


In [ ]:
def mape(y_pred, y_true):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
def train_eval(dataset, model, std, mean, device):
    model.eval()
    loader = torch.utils.data.DataLoader(dataset, batch_size=64) # Use a DataLoader with batch size of 64
    criterion = nn.MSELoss()
    rmse_losses, mae_losses, mape_losses = [], [], []
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            y_pred = (y_pred * std) + mean
            y = (y * std) + mean
            rmse_loss = torch.sqrt(criterion(y_pred, y))
            mae_loss = nn.L1Loss()(y_pred, y)
            mape_loss = mape(y_pred, y) # Check if the `mape()` function is implemented efficiently, as it may cause a bottleneck
            rmse_losses.append(rmse_loss.item())
            mae_losses.append(mae_loss.item())
            mape_losses.append(mape_loss.item())
    return np.mean(rmse_losses), np.mean(mae_losses), np.mean(mape_losses)


def valid_eval(dataset, model, std, mean, device):
    model.eval()
    loader = torch.utils.data.DataLoader(dataset, batch_size=64) # Use a DataLoader with batch size of 64
    criterion = nn.MSELoss()
    rmse_losses, mae_losses, mape_losses = [], [], []
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            y_pred = (y_pred * std) + mean
            y = (y * std) + mean
            rmse_loss = torch.sqrt(criterion(y_pred, y))
            mae_loss = nn.L1Loss()(y_pred, y)
            mape_loss = mape(y_pred, y) # Check if the `mape()` function is implemented efficiently, as it may cause a bottleneck
            rmse_losses.append(rmse_loss.item())
            mae_losses.append(mae_loss.item())
            mape_losses.append(mape_loss.item())
    return np.mean(rmse_losses), np.mean(mae_losses), np.mean(mape_losses)


In [ ]:

@torch.no_grad()
def test_net(loader, model, std, mean, device, args):
    batch_rmse_loss = 0  
    batch_mae_loss = 0
    batch_mape_loss = 0
    data_loader = DataLoader(loader, batch_size=args.batch)
    for idx, (inputs, targets) in enumerate(tqdm(data_loader)):
        model.eval()
        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        
        out_unnorm = output.detach().cpu().numpy()*std + mean
        target_unnorm = targets.detach().cpu().numpy()*std + mean
         
        mae_loss = masked_mae_np(target_unnorm, out_unnorm, 0)
        rmse_loss = masked_rmse_np(target_unnorm, out_unnorm, 0)
        mape_loss = masked_mape_np(target_unnorm, out_unnorm, 0)
        
        batch_rmse_loss += rmse_loss
        batch_mae_loss += mae_loss
        batch_mape_loss += mape_loss

    return batch_rmse_loss / (idx + 1), batch_mae_loss / (idx + 1), batch_mape_loss / (idx + 1)

In [ ]:
for iter in range(args.epochs):
        
        w_locals, loss_locals = [], []
        m = max(int(args.frac * args.num_users), 1)                                              # Sets the max limit for the idx_user
        idxs_users = np.random.choice(range(args.num_users), m, replace=False)                   # Randomly choices value
        
        # print(len(idxs_users))
        # print(idxs_users)
        
        for idx in idxs_users:
            print('Training...')
            print(type(idx))
            local = LocalUpdate(args=args, dataset=train_dataset.dataset, idxs=list(dict_users[idx]))
            net = copy.deepcopy(net_glob).to(DEVICE)


            w, loss = local.train(net)
            w_locals.append(copy.deepcopy(w))
            loss_locals.append(copy.deepcopy(loss))

            # print('Calling eval function...')
            # train_rmse, train_mae, train_mape = train_eval(train_dataset.dataset, net, std, mean, DEVICE)
            # valid_rmse, valid_mae, valid_mape = valid_eval(valloaders.dataset, net, std, mean, DEVICE)


            # if valid_rmse < best_valid_rmse:
            #   best_valid_rmse = valid_rmse
            #   print('New best results!')
            #   torch.save(net.state_dict(), f'net_params_{args.filename}_{args.num_gpu}.pkl')


            # print(f'\n##on train data## loss: {loss}, \n' + 
            #                 f'##on train data## rmse loss: {train_rmse}, mae loss: {train_mae}, mape loss: {train_mape}\n' +
            #                 f'##on valid data## rmse loss: {valid_rmse}, mae loss: {valid_mae}, mape loss: {valid_mape}\n')
                    

        scheduler.step()         # Decays the learning rate of each parameter group by gamma=0.5 every step_size epochs.

        # update global weights
        w_glob = FedAvg(w_locals)

        # copy weight to net_glob
        net_glob.load_state_dict(w_glob)

        # print loss
        loss_avg = sum(loss_locals) / len(loss_locals)
        print('*********************Round {:3d}, Average loss {:.3f}******************************'.format(iter, loss_avg))
        loss_train.append(loss_avg)
        acc_train.append(1-loss_avg)



Training...
<class 'numpy.int64'>
<class 'torch.utils.data.dataloader.DataLoader'>
Update Epoch: 0 [0/6 (0%)]	Loss: 0.472551
*********************Round   0, Average loss 0.473******************************


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [ ]:
test_rmse, test_mae, test_mape = test_net(testloader.dataset, net_glob, std, mean, DEVICE, args)
print(f'##on test data## rmse loss: {test_rmse}, mae loss: {test_mae}, mape loss: {test_mape}')

100%|██████████| 211/211 [00:16<00:00, 13.11it/s]

##on test data## rmse loss: 184.6001821942804, mae loss: 149.38064332934917, mape loss: 209.61457233858334
